In [ ]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, Adadelta, Adagrad
from keras.utils import np_utils, generic_utils
from keras.callbacks import EarlyStopping
from keras.layers.advanced_activations import PReLU, LeakyReLU
from keras.layers import Embedding,GRU,TimeDistributed,RepeatVector,Merge
from keras.preprocessing.text import one_hot
from keras.preprocessing import sequence
import cv2
import numpy as np
from vgg16 import Vgg16

from os import listdir
from os.path import isfile, join

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import PIL.Image

import json
from tqdm import tqdm

from keras.optimizers import SGD, RMSprop, Adam

from utils import *
import cPickle as pickle
from matplotlib import pyplot as plt

import collections


In [ ]:
save_path = "/home/docker/fastai-courses/deeplearning1/nbs/persistent/coco/"
images_path = save_path+"raw_images/val2014"
image_data_arr_path = save_path+"imageDataArr/"
images_concat_t_path = save_path+"imagesConcatT/"
captions_path = save_path+"captions/"
np_save_path = save_path+"temp/"
model_path = save_path+"models/"
images_vgg_features_path = save_path + "images_vgg_features/"


In [ ]:
def get_captions_from_batch(batch_nr):
    captions = pickle.load(open(captions_path+"val2014_captions_"+str(batch_nr)+".p", "rb" ))
    return captions

def get_truncated_captions_from_batch(batch_nr,nr_instances):
    captions = get_captions_from_batch(batch_nr)
    captions = captions[:nr_instances]
    return captions

def first_N_captions_sum(captions,N):
    caption_lengths = [len(caption.split()) for caption in captions]
    counter=collections.Counter(caption_lengths)
    return sum([value for key,value in counter.iteritems() if key <= N])

def get_unique_words(captions):
    unique_words = []
    words = [caption.split() for caption in captions]
   
    for word in words:
        unique_words.extend(word)
        
    unique_words = list(set(unique_words))
    
    return unique_words

In [ ]:
# (40438, 259, 13601)

In [ ]:
captions = get_captions_from_batch(batch_nr = 0)
len(captions)

In [ ]:
totalSum = first_N_captions_sum(captions,500)
totalSum

N = 9
firstN = first_N_captions_sum(captions,N)
print("Captions of length <= %d  ----> Sum = %d  => Percentage = %f"%(N,firstN,firstN/float(totalSum)))

N = 10
firstN = first_N_captions_sum(captions,N)
print("Captions of length <= %d  ---> Sum = %d => Percentage = %f"%(N,firstN,firstN/float(totalSum)))

N = 15
firstN = first_N_captions_sum(captions,N)
print("Captions of length <= %d  ---> Sum = %d => Percentage = %f"%(N,firstN,firstN/float(totalSum)))

N = 20
firstN = first_N_captions_sum(captions,N)
print("Captions of length <= %d  ---> Sum = %d => Percentage = %f"%(N,firstN,firstN/float(totalSum)))

N = 50
firstN = first_N_captions_sum(captions,N)
print("Captions of length <= %d  ---> Sum = %d => Percentage = %f"%(N,firstN,firstN/float(totalSum)))

In [ ]:
def no_words_with_min_appearances(captions,min_no_of_occurences):
    
    sentences = [caption.split() for caption in captions]
    words = []
    for word in sentences:
        words.extend(word)

    counter=collections.Counter(words)
    nr_words = len([_ for word,no_appearances in counter.most_common() if no_appearances >= min_no_of_occurences])
    
    return nr_words

In [ ]:
allWords = float(13601)

In [ ]:
N = 1
nr_words = no_words_with_min_appearances(captions,N)
print("Min_no = %d app --> No of words = %d  ---> Percentage = %f"%(N,nr_words,float(nr_words)/allWords))

N = 5
nr_words = no_words_with_min_appearances(captions,N)
print("Min_no = %d app --> No of words = %d  ---> Percentage = %f"%(N,nr_words,float(nr_words)/allWords))

N = 10
nr_words = no_words_with_min_appearances(captions,N)
print("Min_no = %d app --> No of words = %d  ---> Percentage = %f"%(N,nr_words,float(nr_words)/allWords))

N = 100
nr_words = no_words_with_min_appearances(captions,N)
print("Min_no = %d app --> No of words = %d  ---> Percentage = %f"%(N,nr_words,float(nr_words)/allWords))

N = 500
nr_words = no_words_with_min_appearances(captions,N)
print("Min_no = %d app --> No of words = %d  ---> Percentage = %f"%(N,nr_words,float(nr_words)/allWords))

N = 5000
nr_words = no_words_with_min_appearances(captions,N)
print("Min_no = %d app --> No of words = %d  ---> Percentage = %f"%(N,nr_words,float(nr_words)/allWords))